# My Apple Health
A look into my health data

In [1]:
import xmltodict
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt

Helpful resource: https://www.python-engineer.com/posts/apple-health-data-python/

In [2]:
# plotting defaults
plt.rc('figure', figsize=(20, 7))
plt.style.use('seaborn-whitegrid')
plt.rc('font', size=16)

In [3]:
#file = open("导出.xml",'r')
#contents = file.read()

In [4]:
#input_data = xmltodict.parse(contents)

***Had lots of trouble reading in the data. Apple Health was exporting with some issues. I had to update my iOS to 16.1 and had to delete rows in my xml related to "Hospital Charts"***

In [5]:
#records_list = input_data['HealthData']['Record']

In [6]:
#df = pd.DataFrame(records_list)

In [7]:
#df.to_csv('health_data_2022_12_08.csv',index=False)

In [3]:
df = pd.read_csv('health_data_2022_12_08.csv')

/var/folders/hj/mq308hhs3rz5q1mbkdf0261m0000gn/T/ipykernel_10770/4225088121.py:1: DtypeWarning: Columns (2,7,8,10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('health_data_2022_12_08.csv')


In [4]:
df.info(show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2269523 entries, 0 to 2269522
Data columns (total 11 columns):
 #   Column                            Non-Null Count    Dtype 
---  ------                            --------------    ----- 
 0   @type                             2269523 non-null  object
 1   @sourceName                       2269523 non-null  object
 2   @sourceVersion                    2269484 non-null  object
 3   @unit                             2255241 non-null  object
 4   @creationDate                     2269523 non-null  object
 5   @startDate                        2269523 non-null  object
 6   @endDate                          2269523 non-null  object
 7   @value                            2267590 non-null  object
 8   MetadataEntry                     286013 non-null   object
 9   @device                           2160261 non-null  object
 10  HeartRateVariabilityMetadataList  2587 non-null     object
dtypes: object(11)
memory usage: 190.5+ MB


In [5]:
df.columns.to_list()

['@type',
 '@sourceName',
 '@sourceVersion',
 '@unit',
 '@creationDate',
 '@startDate',
 '@endDate',
 '@value',
 'MetadataEntry',
 '@device',
 'HeartRateVariabilityMetadataList']

In [6]:
#change column names to be more human readable
df = df.rename(columns = {'@type': 'activity_type',
                         '@sourceName': 'source',
                         '@sourceVersion': 'source_ver',
                         '@unit': 'unit_of_measure',
                         '@creationDate': 'creation_time',
                         '@startDate': 'activity_start_time',
                         '@endDate': 'activity_end_time',
                         '@value': 'value',
                         'MetadataEntry': 'metadata',
                         '@device': 'device'})

In [7]:
df.info(show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2269523 entries, 0 to 2269522
Data columns (total 11 columns):
 #   Column                            Non-Null Count    Dtype 
---  ------                            --------------    ----- 
 0   activity_type                     2269523 non-null  object
 1   source                            2269523 non-null  object
 2   source_ver                        2269484 non-null  object
 3   unit_of_measure                   2255241 non-null  object
 4   creation_time                     2269523 non-null  object
 5   activity_start_time               2269523 non-null  object
 6   activity_end_time                 2269523 non-null  object
 7   value                             2267590 non-null  object
 8   metadata                          286013 non-null   object
 9   device                            2160261 non-null  object
 10  HeartRateVariabilityMetadataList  2587 non-null     object
dtypes: object(11)
memory usage: 190.5+ MB


In [8]:
df.drop(columns = ['HeartRateVariabilityMetadataList','metadata'], inplace=True)

In [9]:
df.shape

(2269523, 9)

In [10]:
df['activity_type'].unique()

array(['HKQuantityTypeIdentifierBodyMassIndex',
       'HKQuantityTypeIdentifierHeight',
       'HKQuantityTypeIdentifierBodyMass',
       'HKQuantityTypeIdentifierHeartRate',
       'HKQuantityTypeIdentifierOxygenSaturation',
       'HKQuantityTypeIdentifierBloodPressureSystolic',
       'HKQuantityTypeIdentifierBloodPressureDiastolic',
       'HKQuantityTypeIdentifierRespiratoryRate',
       'HKQuantityTypeIdentifierBodyFatPercentage',
       'HKQuantityTypeIdentifierLeanBodyMass',
       'HKQuantityTypeIdentifierStepCount',
       'HKQuantityTypeIdentifierDistanceWalkingRunning',
       'HKQuantityTypeIdentifierBasalEnergyBurned',
       'HKQuantityTypeIdentifierActiveEnergyBurned',
       'HKQuantityTypeIdentifierFlightsClimbed',
       'HKQuantityTypeIdentifierDietaryFatTotal',
       'HKQuantityTypeIdentifierDietaryFatSaturated',
       'HKQuantityTypeIdentifierDietaryCholesterol',
       'HKQuantityTypeIdentifierDietarySodium',
       'HKQuantityTypeIdentifierDietaryCarbohydrate

In [13]:
# shorter observation names
df['activity_type'] = df['activity_type'].str.replace('HKQuantityTypeIdentifier', '')
df['activity_type'] = df['activity_type'].str.replace('HKCategoryTypeIdentifier', '')
df['activity_type'] = df['activity_type'].str.replace('HKDataType', '')

In [14]:
df['creation_time'] = pd.to_datetime(df['creation_time'])

In [15]:
df = df.set_index('creation_time').sort_index()

In [19]:
china1 = df[:'2016-11-22']

In [22]:
def change_time_zone(df):
    df.tz_convert('Asia/Chongqing')
    df['activity_start_time'] = pd.to_datetime(df['activity_start_time']).dt.tz_convert('Asia/Chongqing')
    df['activity_end_time'] = pd.to_datetime(df['activity_end_time']).dt.tz_convert('Asia/Chongqing')
    return df

In [23]:
change_time_zone(china1)

/var/folders/hj/mq308hhs3rz5q1mbkdf0261m0000gn/T/ipykernel_10770/3623376211.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['activity_start_time'] = pd.to_datetime(df['activity_start_time']).dt.tz_convert('Asia/Chongqing')
/var/folders/hj/mq308hhs3rz5q1mbkdf0261m0000gn/T/ipykernel_10770/3623376211.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['activity_end_time'] = pd.to_datetime(df['activity_end_time']).dt.tz_convert('Asia/Chongqing')


,activity_type,source,source_ver,unit_of_measure,activity_start_time,activity_end_time,value,device
creation_time,,,,,,,,
2015-10-10 06:27:26-06:00,StepCount,Marky Doug,9.0.2,count,2015-10-10 18:21:56+08:00,2015-10-10 18:26:56+08:00,411.0,"<<HKDevice: 0x2833aa8a0>, name:iPhone, manufac..."
2015-10-10 06:27:26-06:00,DistanceWalkingRunning,Marky Doug,9.0.2,km,2015-10-10 18:54:17+08:00,2015-10-10 18:59:18+08:00,0.27712,"<<HKDevice: 0x2833bde00>, name:iPhone, manufac..."
2015-10-10 06:27:26-06:00,DistanceWalkingRunning,Marky Doug,9.0.2,km,2015-10-10 18:48:17+08:00,2015-10-10 18:54:17+08:00,0.00144,"<<HKDevice: 0x2833bde00>, name:iPhone, manufac..."
2015-10-10 06:27:26-06:00,DistanceWalkingRunning,Marky Doug,9.0.2,km,2015-10-10 18:26:56+08:00,2015-10-10 18:31:34+08:00,0.16642,"<<HKDevice: 0x2833bde00>, name:iPhone, manufac..."
2015-10-10 06:27:26-06:00,DistanceWalkingRunning,Marky Doug,9.0.2,km,2015-10-10 18:21:56+08:00,2015-10-10 18:26:56+08:00,0.25881,"<<HKDevice: 0x2833bde00>, name:iPhone, manufac..."
...,...,...,...,...,...,...,...,...
2016-11-22 22:11:46-06:00,StepCount,Marky Doug,10.1.1,count,2016-11-23 12:10:39+08:00,2016-11-23 12:10:44+08:00,22,"<<HKDevice: 0x28337f200>, name:iPhone, manufac..."
2016-11-22 22:30:16-06:00,StepCount,Marky Doug,10.1.1,count,2016-11-23 12:29:09+08:00,2016-11-23 12:29:34+08:00,26,"<<HKDevice: 0x28337f200>, name:iPhone, manufac..."
2016-11-22 23:23:59-06:00,DistanceWalkingRunning,Marky Doug,10.1.1,km,2016-11-23 12:23:09+08:00,2016-11-23 12:29:34+08:00,0.02088,"<<HKDevice: 0x283394be0>, name:iPhone, manufac..."


In [15]:
df

,activity_type,source,source_ver,unit_of_measure,creation_time,activity_start_time,activity_end_time,value,device
0,BodyMassIndex,香山健康,6454,count,2020-05-19 08:21:34+08:00,2020-05-19 08:21:33+08:00,2020-05-19 08:21:33+08:00,27.437,NaN
1,BodyMassIndex,香山健康,6518,count,2020-07-15 07:25:07+08:00,2019-12-15 23:19:12+08:00,2019-12-15 23:19:12+08:00,28.8,NaN
2,BodyMassIndex,香山健康,6518,count,2020-07-15 07:25:07+08:00,2019-12-15 23:19:52+08:00,2019-12-15 23:19:52+08:00,28.6182,NaN
3,BodyMassIndex,香山健康,6518,count,2020-07-15 07:25:07+08:00,2019-12-17 07:54:17+08:00,2019-12-17 07:54:17+08:00,28.3229,NaN
4,BodyMassIndex,香山健康,6518,count,2020-07-15 07:25:07+08:00,2019-12-20 08:49:35+08:00,2019-12-20 08:49:35+08:00,27.9202,NaN
...,...,...,...,...,...,...,...,...,...
2269518,HeartRateVariabilitySDNN,Mark的Apple Watch,9.1,ms,2022-12-08 15:52:33+08:00,2022-12-08 15:51:32+08:00,2022-12-08 15:52:31+08:00,79.6198,"<<HKDevice: 0x2833d1a90>, name:Apple Watch, ma..."
2269519,HeartRateVariabilitySDNN,Mark的Apple Watch,9.1,ms,2022-12-08 17:53:28+08:00,2022-12-08 17:52:27+08:00,2022-12-08 17:53:27+08:00,64.265,"<<HKDevice: 0x2833d1a90>, name:Apple Watch, ma..."
2269520,HeartRateVariabilitySDNN,Mark的Apple Watch,9.1,ms,2022-12-08 19:52:32+08:00,2022-12-08 19:51:31+08:00,2022-12-08 19:52:30+08:00,56.8736,"<<HKDevice: 0x2833d1a90>, name:Apple Watch, ma..."
2269521,HeartRateVariabilitySDNN,Mark的Apple Watch,9.1,ms,2022-12-08 21:52:37+08:00,2022-12-08 21:51:36+08:00,2022-12-08 21:52:36+08:00,36.6104,"<<HKDevice: 0x2833d1a90>, name:Apple Watch, ma..."


In [16]:
df = df.set_index('creation_time').sort_index()

In [35]:
df['activity_start_time'] = df['activity_start_time'].dt.tz_convert('Asia/Chongqing')

In [17]:
df

,activity_type,source,source_ver,unit_of_measure,activity_start_time,activity_end_time,value,device
creation_time,,,,,,,,
2015-10-10 20:27:26+08:00,StepCount,Marky Doug,9.0.2,count,2015-10-10 18:21:56+08:00,2015-10-10 18:26:56+08:00,411.0,"<<HKDevice: 0x2833aa8a0>, name:iPhone, manufac..."
2015-10-10 20:27:26+08:00,DistanceWalkingRunning,Marky Doug,9.0.2,km,2015-10-10 18:54:17+08:00,2015-10-10 18:59:18+08:00,0.27712,"<<HKDevice: 0x2833bde00>, name:iPhone, manufac..."
2015-10-10 20:27:26+08:00,DistanceWalkingRunning,Marky Doug,9.0.2,km,2015-10-10 18:48:17+08:00,2015-10-10 18:54:17+08:00,0.00144,"<<HKDevice: 0x2833bde00>, name:iPhone, manufac..."
2015-10-10 20:27:26+08:00,DistanceWalkingRunning,Marky Doug,9.0.2,km,2015-10-10 18:26:56+08:00,2015-10-10 18:31:34+08:00,0.16642,"<<HKDevice: 0x2833bde00>, name:iPhone, manufac..."
2015-10-10 20:27:26+08:00,DistanceWalkingRunning,Marky Doug,9.0.2,km,2015-10-10 18:21:56+08:00,2015-10-10 18:26:56+08:00,0.25881,"<<HKDevice: 0x2833bde00>, name:iPhone, manufac..."
...,...,...,...,...,...,...,...,...
2022-12-09 02:39:50+08:00,ActiveEnergyBurned,Mark的Apple Watch,9.1,Cal,2022-12-09 02:30:39+08:00,2022-12-09 02:30:49+08:00,0.076,"<<HKDevice: 0x2833b8e10>, name:Apple Watch, ma..."
2022-12-09 02:39:50+08:00,ActiveEnergyBurned,Mark的Apple Watch,9.1,Cal,2022-12-09 02:29:38+08:00,2022-12-09 02:30:39+08:00,1.673,"<<HKDevice: 0x2833b8e10>, name:Apple Watch, ma..."
2022-12-09 02:39:51+08:00,AppleExerciseTime,Mark的Apple Watch,9.1,min,2022-12-09 02:29:00+08:00,2022-12-09 02:30:00+08:00,1.0,"<<HKDevice: 0x2833b9fe0>, name:Apple Watch, ma..."


In [29]:
step_counts = df[df['activity_type'] == 'StepCount']

In [31]:
step_counts.head()

,activity_type,source,source_ver,unit_of_measure,creation_time,activity_start_time,activity_end_time,value,metadata,device
248638,StepCount,Marky Doug,9.0.2,count,2015-10-10 06:27:26-06:00,2015-10-10 00:08:46-06:00,2015-10-10 00:09:06-06:00,16.0,NaN,"<<HKDevice: 0x2833aa8a0>, name:iPhone, manufac..."
248639,StepCount,Marky Doug,9.0.2,count,2015-10-10 06:27:26-06:00,2015-10-10 02:35:59-06:00,2015-10-10 02:41:48-06:00,25.0,NaN,"<<HKDevice: 0x2833aa8a0>, name:iPhone, manufac..."
248640,StepCount,Marky Doug,9.0.2,count,2015-10-10 06:27:26-06:00,2015-10-10 02:41:48-06:00,2015-10-10 02:46:48-06:00,269.0,NaN,"<<HKDevice: 0x2833aa8a0>, name:iPhone, manufac..."
248641,StepCount,Marky Doug,9.0.2,count,2015-10-10 06:27:26-06:00,2015-10-10 02:46:48-06:00,2015-10-10 02:51:50-06:00,569.0,NaN,"<<HKDevice: 0x2833aa8a0>, name:iPhone, manufac..."
248642,StepCount,Marky Doug,9.0.2,count,2015-10-10 06:27:26-06:00,2015-10-10 02:51:50-06:00,2015-10-10 02:54:35-06:00,135.0,NaN,"<<HKDevice: 0x2833aa8a0>, name:iPhone, manufac..."


In [ ]:
step_counts['source'].value_counts().index

In [ ]:
step_counts.info()

In [ ]:
step_counts['value'] = step_counts['value'].astype(int)

In [ ]:
step_counts.info()

In [ ]:
daily_steps = step_counts.resample('D')[['value']].sum()

In [ ]:
#look at all my steps per day
daily_steps.plot()

**Something crazy happened in the middle of 2020. Why did I have over 400,000 steps on a day? Don't think that is possible. *Let's look specifically at those dates to see what's going on***

In [ ]:
daily_steps['2020-05'].plot()

**Looks like something is wrong with around May 24. *Let's go in and look just at that day's data***

In [ ]:
step_counts['2020-05-24']

**There is a new source name here. AND there are over 2000 rows for one day. Maybe there is some duplication. *Let's look at the value counts***

In [ ]:
step_counts['2020-05-24']['sourceName'].value_counts()

In [ ]:
xiao_mi_steps = step_counts[step_counts['source'] == '小米运动']

In [ ]:
xiao_mi_steps.resample('D')[['value']].sum().plot()

In [ ]:
not_xiao_mi = step_counts[step_counts['source'] != '小米运动']

In [ ]:
not_xiao_mi.resample('D')[['value']].sum().plot()

In [ ]:
phone_steps = step_counts[step_counts['source'] == 'Marky Doug']

In [ ]:
phone_steps.resample('D')[['value']].sum().plot()

In [ ]:
apple_watch_steps = step_counts[(step_counts['sourceName'] != 'Marky Doug') & (step_counts['sourceName'] != '小米运动')]

In [ ]:
phone_steps['2021-11':].resample('D')[['value']].sum().plot(title='Phone')
apple_watch_steps.resample('D')[['value']].sum().plot(title='Watch')

In [ ]:
plt.figure(figsize = (20,8))
plt.plot(phone_steps['2021-11':].resample('D')[['value']].sum(), label='Phone', linewidth=1)
plt.plot(apple_watch_steps.resample('D')[['value']].sum(), label='Watch', linewidth=1)
plt.legend()
plt.show()

In [ ]:
apple_watch_steps